In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('whitegrid')
pd.pandas.set_option('display.max_columns', None)

In [2]:
train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')

In [3]:
train['TARGET'].value_counts()

0    282686
1     24825
Name: TARGET, dtype: int64

In [4]:
categorical_features = train.select_dtypes('object')

In [5]:
miss_val = train.isnull().sum()
miss_val_perc = 100 * train.isnull().sum()/len(train)
miss_val_table = pd.concat([miss_val, miss_val_perc], axis = 1)
miss_val_table_rename = miss_val_table.rename(columns = {0: 'Missing Values', 1: '% of Total Values'})
    

In [6]:
miss_val_table_rename[miss_val_table_rename['Missing Values'] != 0].sort_values('% of Total Values', ascending = False).round(1)

,Missing Values,% of Total Values
COMMONAREA_MEDI,214865,69.9
COMMONAREA_AVG,214865,69.9
COMMONAREA_MODE,214865,69.9
NONLIVINGAPARTMENTS_MEDI,213514,69.4
NONLIVINGAPARTMENTS_MODE,213514,69.4
...,...,...
EXT_SOURCE_2,660,0.2
AMT_GOODS_PRICE,278,0.1
AMT_ANNUITY,12,0.0
CNT_FAM_MEMBERS,2,0.0


In [7]:
def missing(df):
    miss_val = df.isnull().sum()
    miss_val_perc = 100 * df.isnull().sum()/len(df)
    miss_val_table = pd.concat([miss_val, miss_val_perc], axis = 1)
    miss_val_table_rename = miss_val_table.rename(columns = {0: 'Missing Values', 1: '% of Total Values'})
    miss = miss_val_table_rename[miss_val_table_rename['Missing Values'] != 0].sort_values('% of Total Values', ascending = False).round(1)
    print("Your dataframe has "+ str(df.shape[1]) + " columns out of which " + str(miss.shape[0]) + " columns have missing values.")
    return miss

In [8]:
missing(train)

Your dataframe has 122 columns out of which 67 columns have missing values.


,Missing Values,% of Total Values
COMMONAREA_MEDI,214865,69.9
COMMONAREA_AVG,214865,69.9
COMMONAREA_MODE,214865,69.9
NONLIVINGAPARTMENTS_MEDI,213514,69.4
NONLIVINGAPARTMENTS_MODE,213514,69.4
...,...,...
EXT_SOURCE_2,660,0.2
AMT_GOODS_PRICE,278,0.1
AMT_ANNUITY,12,0.0
CNT_FAM_MEMBERS,2,0.0


In [9]:
train.dtypes.value_counts()

float64    65
int64      41
object     16
dtype: int64

In [10]:
categorical_features.apply(pd.Series.nunique, axis = 0)

NAME_CONTRACT_TYPE             2
CODE_GENDER                    3
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
NAME_TYPE_SUITE                7
NAME_INCOME_TYPE               8
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
OCCUPATION_TYPE               18
WEEKDAY_APPR_PROCESS_START     7
ORGANIZATION_TYPE             58
FONDKAPREMONT_MODE             4
HOUSETYPE_MODE                 3
WALLSMATERIAL_MODE             7
EMERGENCYSTATE_MODE            2
dtype: int64

In [64]:
bureau = pd.read_csv('../input/home-credit-default-risk/bureau.csv')

In [65]:
previous_loan_accounts = bureau.groupby('SK_ID_CURR', as_index = False)['SK_ID_BUREAU'].count().rename(columns = {"SK_ID_BUREAU": "PREVIOUS_LOAN_ACCOUNTS"})

In [66]:
temp = train.merge(previous_loan_accounts, on = 'SK_ID_CURR', how = 'left')

In [67]:
temp['PREVIOUS_LOAN_ACCOUNTS'].isnull().sum()

44020

In [68]:
temp['PREVIOUS_LOAN_ACCOUNTS'] = temp['PREVIOUS_LOAN_ACCOUNTS'].fillna(0)

In [69]:
bureau_agg = bureau.drop('SK_ID_BUREAU', axis = 1).groupby('SK_ID_CURR', as_index = False).agg(['count','mean', 'median', 'max', 'min','sum', np.std]).reset_index()

In [70]:
columns = ['SK_ID_CURR']
for var in bureau_agg.columns.levels[0]:
    if var != 'SK_ID_CURR':
        for stat in bureau_agg.columns.levels[1][:-1]:
            columns.append('BUREAU_{}_{}'.format(var.upper(), stat.upper()))

In [71]:
bureau_agg.columns = columns

In [72]:
bureau_agg

,SK_ID_CURR,BUREAU_DAYS_CREDIT_COUNT,BUREAU_DAYS_CREDIT_MEAN,BUREAU_DAYS_CREDIT_MEDIAN,BUREAU_DAYS_CREDIT_MAX,BUREAU_DAYS_CREDIT_MIN,BUREAU_DAYS_CREDIT_SUM,BUREAU_DAYS_CREDIT_STD,BUREAU_CREDIT_DAY_OVERDUE_COUNT,BUREAU_CREDIT_DAY_OVERDUE_MEAN,BUREAU_CREDIT_DAY_OVERDUE_MEDIAN,BUREAU_CREDIT_DAY_OVERDUE_MAX,BUREAU_CREDIT_DAY_OVERDUE_MIN,BUREAU_CREDIT_DAY_OVERDUE_SUM,BUREAU_CREDIT_DAY_OVERDUE_STD,BUREAU_DAYS_CREDIT_ENDDATE_COUNT,BUREAU_DAYS_CREDIT_ENDDATE_MEAN,BUREAU_DAYS_CREDIT_ENDDATE_MEDIAN,BUREAU_DAYS_CREDIT_ENDDATE_MAX,BUREAU_DAYS_CREDIT_ENDDATE_MIN,BUREAU_DAYS_CREDIT_ENDDATE_SUM,BUREAU_DAYS_CREDIT_ENDDATE_STD,BUREAU_DAYS_ENDDATE_FACT_COUNT,BUREAU_DAYS_ENDDATE_FACT_MEAN,BUREAU_DAYS_ENDDATE_FACT_MEDIAN,BUREAU_DAYS_ENDDATE_FACT_MAX,BUREAU_DAYS_ENDDATE_FACT_MIN,BUREAU_DAYS_ENDDATE_FACT_SUM,BUREAU_DAYS_ENDDATE_FACT_STD,BUREAU_AMT_CREDIT_MAX_OVERDUE_COUNT,BUREAU_AMT_CREDIT_MAX_OVERDUE_MEAN,BUREAU_AMT_CREDIT_MAX_OVERDUE_MEDIAN,BUREAU_AMT_CREDIT_MAX_OVERDUE_MAX,BUREAU_AMT_CREDIT_MAX_OVERDUE_MIN,BUREAU_AMT_CREDIT_MAX_OVERDUE_SUM,BUREAU_AMT_CREDIT_MAX_OVERDUE_STD,BUREAU_CNT_CREDIT_PROLONG_COUNT,BUREAU_CNT_CREDIT_PROLONG_MEAN,BUREAU_CNT_CREDIT_PROLONG_MEDIAN,BUREAU_CNT_CREDIT_PROLONG_MAX,BUREAU_CNT_CREDIT_PROLONG_MIN,BUREAU_CNT_CREDIT_PROLONG_SUM,BUREAU_CNT_CREDIT_PROLONG_STD,BUREAU_AMT_CREDIT_SUM_COUNT,BUREAU_AMT_CREDIT_SUM_MEAN,BUREAU_AMT_CREDIT_SUM_MEDIAN,BUREAU_AMT_CREDIT_SUM_MAX,BUREAU_AMT_CREDIT_SUM_MIN,BUREAU_AMT_CREDIT_SUM_SUM,BUREAU_AMT_CREDIT_SUM_STD,BUREAU_AMT_CREDIT_SUM_DEBT_COUNT,BUREAU_AMT_CREDIT_SUM_DEBT_MEAN,BUREAU_AMT_CREDIT_SUM_DEBT_MEDIAN,BUREAU_AMT_CREDIT_SUM_DEBT_MAX,BUREAU_AMT_CREDIT_SUM_DEBT_MIN,BUREAU_AMT_CREDIT_SUM_DEBT_SUM,BUREAU_AMT_CREDIT_SUM_DEBT_STD,BUREAU_AMT_CREDIT_SUM_LIMIT_COUNT,BUREAU_AMT_CREDIT_SUM_LIMIT_MEAN,BUREAU_AMT_CREDIT_SUM_LIMIT_MEDIAN,BUREAU_AMT_CREDIT_SUM_LIMIT_MAX,BUREAU_AMT_CREDIT_SUM_LIMIT_MIN,BUREAU_AMT_CREDIT_SUM_LIMIT_SUM,BUREAU_AMT_CREDIT_SUM_LIMIT_STD,BUREAU_AMT_CREDIT_SUM_OVERDUE_COUNT,BUREAU_AMT_CREDIT_SUM_OVERDUE_MEAN,BUREAU_AMT_CREDIT_SUM_OVERDUE_MEDIAN,BUREAU_AMT_CREDIT_SUM_OVERDUE_MAX,BUREAU_AMT_CREDIT_SUM_OVERDUE_MIN,BUREAU_AMT_CREDIT_SUM_OVERDUE_SUM,BUREAU_AMT_CREDIT_SUM_OVERDUE_STD,BUREAU_DAYS_CREDIT_UPDATE_COUNT,BUREAU_DAYS_CREDIT_UPDATE_MEAN,BUREAU_DAYS_CREDIT_UPDATE_MEDIAN,BUREAU_DAYS_CREDIT_UPDATE_MAX,BUREAU_DAYS_CREDIT_UPDATE_MIN,BUREAU_DAYS_CREDIT_UPDATE_SUM,BUREAU_DAYS_CREDIT_UPDATE_STD,BUREAU_AMT_ANNUITY_COUNT,BUREAU_AMT_ANNUITY_MEAN,BUREAU_AMT_ANNUITY_MEDIAN,BUREAU_AMT_ANNUITY_MAX,BUREAU_AMT_ANNUITY_MIN,BUREAU_AMT_ANNUITY_SUM,BUREAU_AMT_ANNUITY_STD
0,100001,7,-735.000000,-857.0,-49,-1572,-5145,489.942514,7,0.0,0.0,0,0,0,0.0,7,82.428571,-179.0,1778.0,-1329.0,577.0,1032.859277,4,-825.500000,-715.0,-544.0,-1328.0,-3302.0,369.078582,0,NaN,NaN,NaN,NaN,0.000,NaN,7,0.000000,0.0,0,0,0,0.000000,7,2.076236e+05,168345.00,378000.00,85500.0,1453365.000,122544.544510,7,85240.928571,0.000,373239.00,0.0,596686.500,137485.631124,6,0.00000,0.0,0.000,0.0,0.000,0.000000,7,0.0,0.0,0.0,0.0,0.0,0.0,7,-93.142857,-155.0,-6,-155,-652,77.204120,7,3545.357143,0.0,10822.5,0.000,24817.500,4800.607529
1,100002,8,-874.000000,-1042.5,-103,-1437,-6992,431.451040,8,0.0,0.0,0,0,0,0.0,6,-349.000000,-424.5,780.0,-1072.0,-2094.0,767.490977,6,-697.500000,-939.0,-36.0,-1185.0,-4185.0,515.992539,5,1681.029,40.500,5043.645,0.0,8405.145,2363.246900,8,0.000000,0.0,0,0,0,0.000000,8,1.081319e+05,54130.50,450000.00,0.0,865055.565,146075.557435,5,49156.200000,0.000,245781.00,0.0,245781.000,109916.604716,4,7997.14125,0.0,31988.565,0.0,31988.565,15994.282500,8,0.0,0.0,0.0,0.0,0.0,0.0,8,-499.875000,-402.5,-7,-1185,-3999,518.522472,7,0.000000,0.0,0.0,0.000,0.000,0.000000
2,100003,4,-1400.750000,-1205.5,-606,-2586,-5603,909.826128,4,0.0,0.0,0,0,0,0.0,4,-544.500000,-480.0,1216.0,-2434.0,-2178.0,1492.770467,3,-1097.333333,-621.0,-540.0,-2131.0,-3292.0,896.097279,4,0.000,0.000,0.000,0.0,0.000,0.000000,4,0.000000,0.0,0,0,0,0.000000,4,2.543501e+05,92576.25,810000.00,22248.0,1017400.500,372269.465535,4,0.000000,0.000,0.00,0.

In [73]:
temp = temp.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')

In [79]:
def agg_fn_numeric(data, col, name):
    df = data.copy()
    for column in df:
        if column != col and 'SK_ID' in column:
            df.drop(column, axis = 1, inplace = True)
    
    group_ids = df[col]
    numeric_df = df.select_dtypes('number')
    numeric_df[col] = group_ids        
    
    agg = numeric_df.groupby(col).agg(['count','mean', 'max', 'min','sum']).reset_index()
    
    c = [col]
    for var in agg.columns.levels[0]:
        if var != col:
            for stat in agg.columns.levels[1][:-1]:
                c.append('{}_{}_{}'.format(name.upper(), var.upper(), stat.upper()))
    agg.columns = c
    return agg

In [80]:
agg_fn_numeric(data = bureau.drop('SK_ID_BUREAU', axis = 1), col = 'SK_ID_CURR', name = 'Bureau')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,SK_ID_CURR,BUREAU_DAYS_CREDIT_COUNT,BUREAU_DAYS_CREDIT_MEAN,BUREAU_DAYS_CREDIT_MAX,BUREAU_DAYS_CREDIT_MIN,BUREAU_DAYS_CREDIT_SUM,BUREAU_CREDIT_DAY_OVERDUE_COUNT,BUREAU_CREDIT_DAY_OVERDUE_MEAN,BUREAU_CREDIT_DAY_OVERDUE_MAX,BUREAU_CREDIT_DAY_OVERDUE_MIN,BUREAU_CREDIT_DAY_OVERDUE_SUM,BUREAU_DAYS_CREDIT_ENDDATE_COUNT,BUREAU_DAYS_CREDIT_ENDDATE_MEAN,BUREAU_DAYS_CREDIT_ENDDATE_MAX,BUREAU_DAYS_CREDIT_ENDDATE_MIN,BUREAU_DAYS_CREDIT_ENDDATE_SUM,BUREAU_DAYS_ENDDATE_FACT_COUNT,BUREAU_DAYS_ENDDATE_FACT_MEAN,BUREAU_DAYS_ENDDATE_FACT_MAX,BUREAU_DAYS_ENDDATE_FACT_MIN,BUREAU_DAYS_ENDDATE_FACT_SUM,BUREAU_AMT_CREDIT_MAX_OVERDUE_COUNT,BUREAU_AMT_CREDIT_MAX_OVERDUE_MEAN,BUREAU_AMT_CREDIT_MAX_OVERDUE_MAX,BUREAU_AMT_CREDIT_MAX_OVERDUE_MIN,BUREAU_AMT_CREDIT_MAX_OVERDUE_SUM,BUREAU_CNT_CREDIT_PROLONG_COUNT,BUREAU_CNT_CREDIT_PROLONG_MEAN,BUREAU_CNT_CREDIT_PROLONG_MAX,BUREAU_CNT_CREDIT_PROLONG_MIN,BUREAU_CNT_CREDIT_PROLONG_SUM,BUREAU_AMT_CREDIT_SUM_COUNT,BUREAU_AMT_CREDIT_SUM_MEAN,BUREAU_AMT_CREDIT_SUM_MAX,BUREAU_AMT_CREDIT_SUM_MIN,BUREAU_AMT_CREDIT_SUM_SUM,BUREAU_AMT_CREDIT_SUM_DEBT_COUNT,BUREAU_AMT_CREDIT_SUM_DEBT_MEAN,BUREAU_AMT_CREDIT_SUM_DEBT_MAX,BUREAU_AMT_CREDIT_SUM_DEBT_MIN,BUREAU_AMT_CREDIT_SUM_DEBT_SUM,BUREAU_AMT_CREDIT_SUM_LIMIT_COUNT,BUREAU_AMT_CREDIT_SUM_LIMIT_MEAN,BUREAU_AMT_CREDIT_SUM_LIMIT_MAX,BUREAU_AMT_CREDIT_SUM_LIMIT_MIN,BUREAU_AMT_CREDIT_SUM_LIMIT_SUM,BUREAU_AMT_CREDIT_SUM_OVERDUE_COUNT,BUREAU_AMT_CREDIT_SUM_OVERDUE_MEAN,BUREAU_AMT_CREDIT_SUM_OVERDUE_MAX,BUREAU_AMT_CREDIT_SUM_OVERDUE_MIN,BUREAU_AMT_CREDIT_SUM_OVERDUE_SUM,BUREAU_DAYS_CREDIT_UPDATE_COUNT,BUREAU_DAYS_CREDIT_UPDATE_MEAN,BUREAU_DAYS_CREDIT_UPDATE_MAX,BUREAU_DAYS_CREDIT_UPDATE_MIN,BUREAU_DAYS_CREDIT_UPDATE_SUM,BUREAU_AMT_ANNUITY_COUNT,BUREAU_AMT_ANNUITY_MEAN,BUREAU_AMT_ANNUITY_MAX,BUREAU_AMT_ANNUITY_MIN,BUREAU_AMT_ANNUITY_SUM
0,100001,7,-735.000000,-49,-1572,-5145,7,0.0,0,0,0,7,82.428571,1778.0,-1329.0,577.0,4,-825.500000,-544.0,-1328.0,-3302.0,0,NaN,NaN,NaN,0.000,7,0.000000,0,0,0,7,2.076236e+05,378000.00,85500.0,1453365.000,7,85240.928571,373239.00,0.0,596686.500,6,0.00000,0.000,0.0,0.000,7,0.0,0.0,0.0,0.0,7,-93.142857,-6,-155,-652,7,3545.357143,10822.5,0.000,24817.500
1,100002,8,-874.000000,-103,-1437,-6992,8,0.0,0,0,0,6,-349.000000,780.0,-1072.0,-2094.0,6,-697.500000,-36.0,-1185.0,-4185.0,5,1681.029,5043.645,0.0,8405.145,8,0.000000,0,0,0,8,1.081319e+05,450000.00,0.0,865055.565,5,49156.200000,245781.00,0.0,245781.000,4,7997.14125,31988.565,0.0,31988.565,8,0.0,0.0,0.0,0.0,8,-499.875000,-7,-1185,-3999,7,0.000000,0.0,0.000,0.000
2,100003,4,-1400.750000,-606,-2586,-5603,4,0.0,0,0,0,4,-544.500000,1216.0,-2434.0,-2178.0,3,-1097.333333,-540.0,-2131.0,-3292.0,4,0.000,0.000,0.0,0.000,4,0.000000,0,0,0,4,2.543501e+05,810000.00,22248.0,1017400.500,4,0.000000,0.00,0.0,0.000,4,202500.00000,810000.000,0.0,810000.000,4,0.0,0.0,0.0,0.0,4,-816.000000,-43,-2131,-3264,0,NaN,NaN,NaN,0.000
3,100004,2,-867.000000,-408,-1326,-1734,2,0.0,0,0,0,2,-488.500000,-382.0,-595.0,-977.0,2,-532.500000,-382.0,-683.0,-1065.0,1,0.000,0.000,0.0,0.000,2,0.000000,0,0,0,2,9.451890e+04,94537.80,94500.0,189037.800,2,0.000000,0.00,0.0,0.000,2,0.00000,0.000,0.0,0.000,2,0.0,0.0,0.0,0.0,2,-532.000000,-382,-682,-1064,0,NaN,NaN,NaN,0.000
4,100005,3,-190.666667,-62,-373,-572,3,0.0,0,0,0,3,439.333333,1324.0,-128.0,1318.0,1,-123.000000,-123.0,-123.0,-123.0,1,0.000,0.000,0.0,0.000,3,0.000000,0,0,0,3,2.190420e+05,568800.00,29826.0,657126.000,3,189469.500000,543087.00,0.0,568408.500,3,0.00000,0.000,0.0,0.000,3,0.0,0.0,0.0,0.0,3,-54.333333,-11,-121,-163,3,1420.500000,4261.5,0.000,4261.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305806,456249,13,-1667.076923,-483,-2713,-21672,13,0.0,0,0,0,12,-1232.333333,1363.0,-2499.0,-14788.0,12,-1364.750000,-291.0,-2525.0,-16377.0,5,6147.000,18945.000,0.0,30735.000,13,0.000000,0,0,0,13

In [81]:
bureau_agg

,SK_ID_CURR,BUREAU_DAYS_CREDIT_COUNT,BUREAU_DAYS_CREDIT_MEAN,BUREAU_DAYS_CREDIT_MEDIAN,BUREAU_DAYS_CREDIT_MAX,BUREAU_DAYS_CREDIT_MIN,BUREAU_DAYS_CREDIT_SUM,BUREAU_DAYS_CREDIT_STD,BUREAU_CREDIT_DAY_OVERDUE_COUNT,BUREAU_CREDIT_DAY_OVERDUE_MEAN,BUREAU_CREDIT_DAY_OVERDUE_MEDIAN,BUREAU_CREDIT_DAY_OVERDUE_MAX,BUREAU_CREDIT_DAY_OVERDUE_MIN,BUREAU_CREDIT_DAY_OVERDUE_SUM,BUREAU_CREDIT_DAY_OVERDUE_STD,BUREAU_DAYS_CREDIT_ENDDATE_COUNT,BUREAU_DAYS_CREDIT_ENDDATE_MEAN,BUREAU_DAYS_CREDIT_ENDDATE_MEDIAN,BUREAU_DAYS_CREDIT_ENDDATE_MAX,BUREAU_DAYS_CREDIT_ENDDATE_MIN,BUREAU_DAYS_CREDIT_ENDDATE_SUM,BUREAU_DAYS_CREDIT_ENDDATE_STD,BUREAU_DAYS_ENDDATE_FACT_COUNT,BUREAU_DAYS_ENDDATE_FACT_MEAN,BUREAU_DAYS_ENDDATE_FACT_MEDIAN,BUREAU_DAYS_ENDDATE_FACT_MAX,BUREAU_DAYS_ENDDATE_FACT_MIN,BUREAU_DAYS_ENDDATE_FACT_SUM,BUREAU_DAYS_ENDDATE_FACT_STD,BUREAU_AMT_CREDIT_MAX_OVERDUE_COUNT,BUREAU_AMT_CREDIT_MAX_OVERDUE_MEAN,BUREAU_AMT_CREDIT_MAX_OVERDUE_MEDIAN,BUREAU_AMT_CREDIT_MAX_OVERDUE_MAX,BUREAU_AMT_CREDIT_MAX_OVERDUE_MIN,BUREAU_AMT_CREDIT_MAX_OVERDUE_SUM,BUREAU_AMT_CREDIT_MAX_OVERDUE_STD,BUREAU_CNT_CREDIT_PROLONG_COUNT,BUREAU_CNT_CREDIT_PROLONG_MEAN,BUREAU_CNT_CREDIT_PROLONG_MEDIAN,BUREAU_CNT_CREDIT_PROLONG_MAX,BUREAU_CNT_CREDIT_PROLONG_MIN,BUREAU_CNT_CREDIT_PROLONG_SUM,BUREAU_CNT_CREDIT_PROLONG_STD,BUREAU_AMT_CREDIT_SUM_COUNT,BUREAU_AMT_CREDIT_SUM_MEAN,BUREAU_AMT_CREDIT_SUM_MEDIAN,BUREAU_AMT_CREDIT_SUM_MAX,BUREAU_AMT_CREDIT_SUM_MIN,BUREAU_AMT_CREDIT_SUM_SUM,BUREAU_AMT_CREDIT_SUM_STD,BUREAU_AMT_CREDIT_SUM_DEBT_COUNT,BUREAU_AMT_CREDIT_SUM_DEBT_MEAN,BUREAU_AMT_CREDIT_SUM_DEBT_MEDIAN,BUREAU_AMT_CREDIT_SUM_DEBT_MAX,BUREAU_AMT_CREDIT_SUM_DEBT_MIN,BUREAU_AMT_CREDIT_SUM_DEBT_SUM,BUREAU_AMT_CREDIT_SUM_DEBT_STD,BUREAU_AMT_CREDIT_SUM_LIMIT_COUNT,BUREAU_AMT_CREDIT_SUM_LIMIT_MEAN,BUREAU_AMT_CREDIT_SUM_LIMIT_MEDIAN,BUREAU_AMT_CREDIT_SUM_LIMIT_MAX,BUREAU_AMT_CREDIT_SUM_LIMIT_MIN,BUREAU_AMT_CREDIT_SUM_LIMIT_SUM,BUREAU_AMT_CREDIT_SUM_LIMIT_STD,BUREAU_AMT_CREDIT_SUM_OVERDUE_COUNT,BUREAU_AMT_CREDIT_SUM_OVERDUE_MEAN,BUREAU_AMT_CREDIT_SUM_OVERDUE_MEDIAN,BUREAU_AMT_CREDIT_SUM_OVERDUE_MAX,BUREAU_AMT_CREDIT_SUM_OVERDUE_MIN,BUREAU_AMT_CREDIT_SUM_OVERDUE_SUM,BUREAU_AMT_CREDIT_SUM_OVERDUE_STD,BUREAU_DAYS_CREDIT_UPDATE_COUNT,BUREAU_DAYS_CREDIT_UPDATE_MEAN,BUREAU_DAYS_CREDIT_UPDATE_MEDIAN,BUREAU_DAYS_CREDIT_UPDATE_MAX,BUREAU_DAYS_CREDIT_UPDATE_MIN,BUREAU_DAYS_CREDIT_UPDATE_SUM,BUREAU_DAYS_CREDIT_UPDATE_STD,BUREAU_AMT_ANNUITY_COUNT,BUREAU_AMT_ANNUITY_MEAN,BUREAU_AMT_ANNUITY_MEDIAN,BUREAU_AMT_ANNUITY_MAX,BUREAU_AMT_ANNUITY_MIN,BUREAU_AMT_ANNUITY_SUM,BUREAU_AMT_ANNUITY_STD
0,100001,7,-735.000000,-857.0,-49,-1572,-5145,489.942514,7,0.0,0.0,0,0,0,0.0,7,82.428571,-179.0,1778.0,-1329.0,577.0,1032.859277,4,-825.500000,-715.0,-544.0,-1328.0,-3302.0,369.078582,0,NaN,NaN,NaN,NaN,0.000,NaN,7,0.000000,0.0,0,0,0,0.000000,7,2.076236e+05,168345.00,378000.00,85500.0,1453365.000,122544.544510,7,85240.928571,0.000,373239.00,0.0,596686.500,137485.631124,6,0.00000,0.0,0.000,0.0,0.000,0.000000,7,0.0,0.0,0.0,0.0,0.0,0.0,7,-93.142857,-155.0,-6,-155,-652,77.204120,7,3545.357143,0.0,10822.5,0.000,24817.500,4800.607529
1,100002,8,-874.000000,-1042.5,-103,-1437,-6992,431.451040,8,0.0,0.0,0,0,0,0.0,6,-349.000000,-424.5,780.0,-1072.0,-2094.0,767.490977,6,-697.500000,-939.0,-36.0,-1185.0,-4185.0,515.992539,5,1681.029,40.500,5043.645,0.0,8405.145,2363.246900,8,0.000000,0.0,0,0,0,0.000000,8,1.081319e+05,54130.50,450000.00,0.0,865055.565,146075.557435,5,49156.200000,0.000,245781.00,0.0,245781.000,109916.604716,4,7997.14125,0.0,31988.565,0.0,31988.565,15994.282500,8,0.0,0.0,0.0,0.0,0.0,0.0,8,-499.875000,-402.5,-7,-1185,-3999,518.522472,7,0.000000,0.0,0.0,0.000,0.000,0.000000
2,100003,4,-1400.750000,-1205.5,-606,-2586,-5603,909.826128,4,0.0,0.0,0,0,0,0.0,4,-544.500000,-480.0,1216.0,-2434.0,-2178.0,1492.770467,3,-1097.333333,-621.0,-540.0,-2131.0,-3292.0,896.097279,4,0.000,0.000,0.000,0.0,0.000,0.000000,4,0.000000,0.0,0,0,0,0.000000,4,2.543501e+05,92576.25,810000.00,22248.0,1017400.500,372269.465535,4,0.000000,0.000,0.00,0.

In [82]:
categorical = pd.get_dummies(bureau.select_dtypes('object'))
categorical['SK_ID_CURR'] = bureau['SK_ID_CURR']

In [83]:
categorical_grouped = categorical.groupby('SK_ID_CURR').agg(['sum', 'mean'])

In [84]:
categorical_grouped

CREDIT_ACTIVE_Active           CREDIT_ACTIVE_Bad debt       \
                            sum      mean                    sum mean   
SK_ID_CURR                                                              
100001                        3  0.428571                      0  0.0   
100002                        2  0.250000                      0  0.0   
100003                        1  0.250000                      0  0.0   
100004                        0  0.000000                      0  0.0   
100005                        2  0.666667                      0  0.0   
...                         ...       ...                    ...  ...   
456249                        2  0.153846                      0  0.0   
456250                        2  0.666667                      0  0.0   
456253                        2  0.500000                      0  0.0   
456254                        0  0.000000                      0  0.0   
456255                        5  0.454545                      0  0.0   

           CREDIT_ACTIVE_Closed           CREDIT_ACTIVE_Sold       \
                            sum      mean                sum mean   
SK_ID_CURR                                                          
100001                        4  0.571429                  0  0.0   
100002                        6  0.750000                  0  0.0   
100003                        3  0.750000                  0  0.0   
100004                        2  1.000000                  0  0.0   
100005                        1  0.333333                  0  0.0   
...                         ...       ...                ...  ...   
456249                       11  0.846154                  0  0.0   
456250                        1  0.333333                  0  0.0   
456253                        2  0.500000                  0  0.0   
456254                        1  1.000000                  0  0.0   
456255                        6  0.545455                  0  0.0   

           CREDIT_CURRENCY_currency 1      CREDIT_CURRENCY_currency 2       \
                                  sum mean                        sum mean   
SK_ID_CURR                                                                   
100001                              7  1.0                          0  0.0   
100002                              8  1.0                          0  0.0   
100003                              4  1.0                          0  0.0   
100004                              2  1.0                          0  0.0   
100005                              3  1.0                          0  0.0   
...                               ...  ...                        ...  ...   
456249                             13  1.0                          0  0.0   
456250                              3  1.0                          0  0.0   
456253                              4  1.0                          0  0.0   
456254                              1  1.0                          0  0.0   
456255                             11  1.0                          0  0.0   

           CREDIT_CURRENCY_currency 3      CREDIT_CURRENCY_currency 4       \
                                  sum mean                        sum mean   
SK_ID_CURR                                                                   
100001                              0  0.0                          0  0.0   
100002                              0  0.0                          0  0.0   
100003                              0  0.0                          0  0.0   
100004                              0  0.0                          0  0.0   
100005                              0  0.0                          0  0.0   
...                               ...  ...                        ...  ...   
456249                              0  0.0                          0  0.0   
456250                              0  0.0                          0  0.0   
456253                              0  0.0                          0 

In [85]:
categorical_grouped.columns.levels[0]

Index(['CREDIT_ACTIVE_Active', 'CREDIT_ACTIVE_Bad debt',
       'CREDIT_ACTIVE_Closed', 'CREDIT_ACTIVE_Sold',
       'CREDIT_CURRENCY_currency 1', 'CREDIT_CURRENCY_currency 2',
       'CREDIT_CURRENCY_currency 3', 'CREDIT_CURRENCY_currency 4',
       'CREDIT_TYPE_Another type of loan', 'CREDIT_TYPE_Car loan',
       'CREDIT_TYPE_Cash loan (non-earmarked)', 'CREDIT_TYPE_Consumer credit',
       'CREDIT_TYPE_Credit card', 'CREDIT_TYPE_Interbank credit',
       'CREDIT_TYPE_Loan for business development',
       'CREDIT_TYPE_Loan for purchase of shares (margin lending)',
       'CREDIT_TYPE_Loan for the purchase of equipment',
       'CREDIT_TYPE_Loan for working capital replenishment',
       'CREDIT_TYPE_Microloan', 'CREDIT_TYPE_Mobile operator loan',
       'CREDIT_TYPE_Mortgage', 'CREDIT_TYPE_Real estate loan',
       'CREDIT_TYPE_Unknown type of loan'],
      dtype='object')

In [86]:
gr = 'SK_ID_CURR'
columns = []
for col in categorical_grouped.columns.levels[0]:
    if col != gr:
        for stat in ['COUNT', 'COUNT_NORM']:
            columns.append('{}_{}'.format(col.upper(), stat.upper()))

In [87]:
categorical_grouped.columns = columns

In [88]:
categorical_grouped

,CREDIT_ACTIVE_ACTIVE_COUNT,CREDIT_ACTIVE_ACTIVE_COUNT_NORM,CREDIT_ACTIVE_BAD DEBT_COUNT,CREDIT_ACTIVE_BAD DEBT_COUNT_NORM,CREDIT_ACTIVE_CLOSED_COUNT,CREDIT_ACTIVE_CLOSED_COUNT_NORM,CREDIT_ACTIVE_SOLD_COUNT,CREDIT_ACTIVE_SOLD_COUNT_NORM,CREDIT_CURRENCY_CURRENCY 1_COUNT,CREDIT_CURRENCY_CURRENCY 1_COUNT_NORM,CREDIT_CURRENCY_CURRENCY 2_COUNT,CREDIT_CURRENCY_CURRENCY 2_COUNT_NORM,CREDIT_CURRENCY_CURRENCY 3_COUNT,CREDIT_CURRENCY_CURRENCY 3_COUNT_NORM,CREDIT_CURRENCY_CURRENCY 4_COUNT,CREDIT_CURRENCY_CURRENCY 4_COUNT_NORM,CREDIT_TYPE_ANOTHER TYPE OF LOAN_COUNT,CREDIT_TYPE_ANOTHER TYPE OF LOAN_COUNT_NORM,CREDIT_TYPE_CAR LOAN_COUNT,CREDIT_TYPE_CAR LOAN_COUNT_NORM,CREDIT_TYPE_CASH LOAN (NON-EARMARKED)_COUNT,CREDIT_TYPE_CASH LOAN (NON-EARMARKED)_COUNT_NORM,CREDIT_TYPE_CONSUMER CREDIT_COUNT,CREDIT_TYPE_CONSUMER CREDIT_COUNT_NORM,CREDIT_TYPE_CREDIT CARD_COUNT,CREDIT_TYPE_CREDIT CARD_COUNT_NORM,CREDIT_TYPE_INTERBANK CREDIT_COUNT,CREDIT_TYPE_INTERBANK CREDIT_COUNT_NORM,CREDIT_TYPE_LOAN FOR BUSINESS DEVELOPMENT_COUNT,CREDIT_TYPE_LOAN FOR BUSINESS DEVELOPMENT_COUNT_NORM,CREDIT_TYPE_LOAN FOR PURCHASE OF SHARES (MARGIN LENDING)_COUNT,CREDIT_TYPE_LOAN FOR PURCHASE OF SHARES (MARGIN LENDING)_COUNT_NORM,CREDIT_TYPE_LOAN FOR THE PURCHASE OF EQUIPMENT_COUNT,CREDIT_TYPE_LOAN FOR THE PURCHASE OF EQUIPMENT_COUNT_NORM,CREDIT_TYPE_LOAN FOR WORKING CAPITAL REPLENISHMENT_COUNT,CREDIT_TYPE_LOAN FOR WORKING CAPITAL REPLENISHMENT_COUNT_NORM,CREDIT_TYPE_MICROLOAN_COUNT,CREDIT_TYPE_MICROLOAN_COUNT_NORM,CREDIT_TYPE_MOBILE OPERATOR LOAN_COUNT,CREDIT_TYPE_MOBILE OPERATOR LOAN_COUNT_NORM,CREDIT_TYPE_MORTGAGE_COUNT,CREDIT_TYPE_MORTGAGE_COUNT_NORM,CREDIT_TYPE_REAL ESTATE LOAN_COUNT,CREDIT_TYPE_REAL ESTATE LOAN_COUNT_NORM,CREDIT_TYPE_UNKNOWN TYPE OF LOAN_COUNT,CREDIT_TYPE_UNKNOWN TYPE OF LOAN_COUNT_NORM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,3,0.428571,0,0.0,4,0.571429,0,0.0,7,1.0,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,7,1.000000,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100002,2,0.250000,0,0.0,6,0.750000,0,0.0,8,1.0,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,4,0.500000,4,0.500000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100003,1,0.250000,0,0.0,3,0.750000,0,0.0,4,1.0,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,2,0.500000,2,0.500000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100004,0,0.000000,0,0.0,2,1.000000,0,0.0,2,1.0,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,2,1.000000,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100005,2,0.666667,0,0.0,1,0.333333,0,0.0,3,1.0,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,2,0.666667,1,0.333333,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456249,2,0.153846,0,0.0,11,0.846154,0,0.0,13,1.0,0,0.0,0,0.0,0,0.0,1,0.076923,0,0.0,0,0.0,9,0.692308,3,0.230769,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
456250,2,0.666667,0,0.0,1,0.333333,0,0.0,3,1.0,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,2,0.666667,1,0.333333,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
456253,2,0.500000,0,0.0,2,0.500000,0,0.0,4,1.0,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,3,0.750000,1,0.250000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


In [89]:
def agg_fn_categorical(data, col, name):
    df = data.copy()
    for column in df:
        if column != col and 'SK_ID' in column:
            df.drop(column, axis = 1, inplace = True)
        
    group_ids = df[col]
    categorical_df = pd.get_dummies(df.select_dtypes('object'))
    categorical_df[col] = group_ids 

    agg = categorical_df.groupby(col).agg(['sum', 'mean'])
    
    columns = []
    for var in agg.columns.levels[0]:
        for stat in ['COUNT', 'COUNT_NORM']:
            columns.append('{}_{}_{}'.format(name.upper(), var.upper(), stat))

    agg.columns = columns
    return agg



In [90]:
agg_fn_categorical(bureau, 'SK_ID_CURR', 'BUREAU')

,BUREAU_CREDIT_ACTIVE_ACTIVE_COUNT,BUREAU_CREDIT_ACTIVE_ACTIVE_COUNT_NORM,BUREAU_CREDIT_ACTIVE_BAD DEBT_COUNT,BUREAU_CREDIT_ACTIVE_BAD DEBT_COUNT_NORM,BUREAU_CREDIT_ACTIVE_CLOSED_COUNT,BUREAU_CREDIT_ACTIVE_CLOSED_COUNT_NORM,BUREAU_CREDIT_ACTIVE_SOLD_COUNT,BUREAU_CREDIT_ACTIVE_SOLD_COUNT_NORM,BUREAU_CREDIT_CURRENCY_CURRENCY 1_COUNT,BUREAU_CREDIT_CURRENCY_CURRENCY 1_COUNT_NORM,BUREAU_CREDIT_CURRENCY_CURRENCY 2_COUNT,BUREAU_CREDIT_CURRENCY_CURRENCY 2_COUNT_NORM,BUREAU_CREDIT_CURRENCY_CURRENCY 3_COUNT,BUREAU_CREDIT_CURRENCY_CURRENCY 3_COUNT_NORM,BUREAU_CREDIT_CURRENCY_CURRENCY 4_COUNT,BUREAU_CREDIT_CURRENCY_CURRENCY 4_COUNT_NORM,BUREAU_CREDIT_TYPE_ANOTHER TYPE OF LOAN_COUNT,BUREAU_CREDIT_TYPE_ANOTHER TYPE OF LOAN_COUNT_NORM,BUREAU_CREDIT_TYPE_CAR LOAN_COUNT,BUREAU_CREDIT_TYPE_CAR LOAN_COUNT_NORM,BUREAU_CREDIT_TYPE_CASH LOAN (NON-EARMARKED)_COUNT,BUREAU_CREDIT_TYPE_CASH LOAN (NON-EARMARKED)_COUNT_NORM,BUREAU_CREDIT_TYPE_CONSUMER CREDIT_COUNT,BUREAU_CREDIT_TYPE_CONSUMER CREDIT_COUNT_NORM,BUREAU_CREDIT_TYPE_CREDIT CARD_COUNT,BUREAU_CREDIT_TYPE_CREDIT CARD_COUNT_NORM,BUREAU_CREDIT_TYPE_INTERBANK CREDIT_COUNT,BUREAU_CREDIT_TYPE_INTERBANK CREDIT_COUNT_NORM,BUREAU_CREDIT_TYPE_LOAN FOR BUSINESS DEVELOPMENT_COUNT,BUREAU_CREDIT_TYPE_LOAN FOR BUSINESS DEVELOPMENT_COUNT_NORM,BUREAU_CREDIT_TYPE_LOAN FOR PURCHASE OF SHARES (MARGIN LENDING)_COUNT,BUREAU_CREDIT_TYPE_LOAN FOR PURCHASE OF SHARES (MARGIN LENDING)_COUNT_NORM,BUREAU_CREDIT_TYPE_LOAN FOR THE PURCHASE OF EQUIPMENT_COUNT,BUREAU_CREDIT_TYPE_LOAN FOR THE PURCHASE OF EQUIPMENT_COUNT_NORM,BUREAU_CREDIT_TYPE_LOAN FOR WORKING CAPITAL REPLENISHMENT_COUNT,BUREAU_CREDIT_TYPE_LOAN FOR WORKING CAPITAL REPLENISHMENT_COUNT_NORM,BUREAU_CREDIT_TYPE_MICROLOAN_COUNT,BUREAU_CREDIT_TYPE_MICROLOAN_COUNT_NORM,BUREAU_CREDIT_TYPE_MOBILE OPERATOR LOAN_COUNT,BUREAU_CREDIT_TYPE_MOBILE OPERATOR LOAN_COUNT_NORM,BUREAU_CREDIT_TYPE_MORTGAGE_COUNT,BUREAU_CREDIT_TYPE_MORTGAGE_COUNT_NORM,BUREAU_CREDIT_TYPE_REAL ESTATE LOAN_COUNT,BUREAU_CREDIT_TYPE_REAL ESTATE LOAN_COUNT_NORM,BUREAU_CREDIT_TYPE_UNKNOWN TYPE OF LOAN_COUNT,BUREAU_CREDIT_TYPE_UNKNOWN TYPE OF LOAN_COUNT_NORM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,3,0.428571,0,0.0,4,0.571429,0,0.0,7,1.0,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,7,1.000000,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100002,2,0.250000,0,0.0,6,0.750000,0,0.0,8,1.0,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,4,0.500000,4,0.500000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100003,1,0.250000,0,0.0,3,0.750000,0,0.0,4,1.0,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,2,0.500000,2,0.500000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100004,0,0.000000,0,0.0,2,1.000000,0,0.0,2,1.0,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,2,1.000000,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100005,2,0.666667,0,0.0,1,0.333333,0,0.0,3,1.0,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,2,0.666667,1,0.333333,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456249,2,0.153846,0,0.0,11,0.846154,0,0.0,13,1.0,0,0.0,0,0.0,0,0.0,1,0.076923,0,0.0,0,0.0,9,0.692308,3,0.230769,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
456250,2,0.666667,0,0.0,1,0.333333,0,0.0,3,1.0,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,2,0.666667,1,0.333333,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
456253,2,0.500000,0,0.0,2,0.500000,0,0.0,4,1.0,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,3,0.750000,1,0.250000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


In [91]:
categorical_grouped

,CREDIT_ACTIVE_ACTIVE_COUNT,CREDIT_ACTIVE_ACTIVE_COUNT_NORM,CREDIT_ACTIVE_BAD DEBT_COUNT,CREDIT_ACTIVE_BAD DEBT_COUNT_NORM,CREDIT_ACTIVE_CLOSED_COUNT,CREDIT_ACTIVE_CLOSED_COUNT_NORM,CREDIT_ACTIVE_SOLD_COUNT,CREDIT_ACTIVE_SOLD_COUNT_NORM,CREDIT_CURRENCY_CURRENCY 1_COUNT,CREDIT_CURRENCY_CURRENCY 1_COUNT_NORM,CREDIT_CURRENCY_CURRENCY 2_COUNT,CREDIT_CURRENCY_CURRENCY 2_COUNT_NORM,CREDIT_CURRENCY_CURRENCY 3_COUNT,CREDIT_CURRENCY_CURRENCY 3_COUNT_NORM,CREDIT_CURRENCY_CURRENCY 4_COUNT,CREDIT_CURRENCY_CURRENCY 4_COUNT_NORM,CREDIT_TYPE_ANOTHER TYPE OF LOAN_COUNT,CREDIT_TYPE_ANOTHER TYPE OF LOAN_COUNT_NORM,CREDIT_TYPE_CAR LOAN_COUNT,CREDIT_TYPE_CAR LOAN_COUNT_NORM,CREDIT_TYPE_CASH LOAN (NON-EARMARKED)_COUNT,CREDIT_TYPE_CASH LOAN (NON-EARMARKED)_COUNT_NORM,CREDIT_TYPE_CONSUMER CREDIT_COUNT,CREDIT_TYPE_CONSUMER CREDIT_COUNT_NORM,CREDIT_TYPE_CREDIT CARD_COUNT,CREDIT_TYPE_CREDIT CARD_COUNT_NORM,CREDIT_TYPE_INTERBANK CREDIT_COUNT,CREDIT_TYPE_INTERBANK CREDIT_COUNT_NORM,CREDIT_TYPE_LOAN FOR BUSINESS DEVELOPMENT_COUNT,CREDIT_TYPE_LOAN FOR BUSINESS DEVELOPMENT_COUNT_NORM,CREDIT_TYPE_LOAN FOR PURCHASE OF SHARES (MARGIN LENDING)_COUNT,CREDIT_TYPE_LOAN FOR PURCHASE OF SHARES (MARGIN LENDING)_COUNT_NORM,CREDIT_TYPE_LOAN FOR THE PURCHASE OF EQUIPMENT_COUNT,CREDIT_TYPE_LOAN FOR THE PURCHASE OF EQUIPMENT_COUNT_NORM,CREDIT_TYPE_LOAN FOR WORKING CAPITAL REPLENISHMENT_COUNT,CREDIT_TYPE_LOAN FOR WORKING CAPITAL REPLENISHMENT_COUNT_NORM,CREDIT_TYPE_MICROLOAN_COUNT,CREDIT_TYPE_MICROLOAN_COUNT_NORM,CREDIT_TYPE_MOBILE OPERATOR LOAN_COUNT,CREDIT_TYPE_MOBILE OPERATOR LOAN_COUNT_NORM,CREDIT_TYPE_MORTGAGE_COUNT,CREDIT_TYPE_MORTGAGE_COUNT_NORM,CREDIT_TYPE_REAL ESTATE LOAN_COUNT,CREDIT_TYPE_REAL ESTATE LOAN_COUNT_NORM,CREDIT_TYPE_UNKNOWN TYPE OF LOAN_COUNT,CREDIT_TYPE_UNKNOWN TYPE OF LOAN_COUNT_NORM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,3,0.428571,0,0.0,4,0.571429,0,0.0,7,1.0,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,7,1.000000,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100002,2,0.250000,0,0.0,6,0.750000,0,0.0,8,1.0,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,4,0.500000,4,0.500000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100003,1,0.250000,0,0.0,3,0.750000,0,0.0,4,1.0,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,2,0.500000,2,0.500000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100004,0,0.000000,0,0.0,2,1.000000,0,0.0,2,1.0,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,2,1.000000,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100005,2,0.666667,0,0.0,1,0.333333,0,0.0,3,1.0,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,2,0.666667,1,0.333333,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456249,2,0.153846,0,0.0,11,0.846154,0,0.0,13,1.0,0,0.0,0,0.0,0,0.0,1,0.076923,0,0.0,0,0.0,9,0.692308,3,0.230769,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
456250,2,0.666667,0,0.0,1,0.333333,0,0.0,3,1.0,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,2,0.666667,1,0.333333,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
456253,2,0.500000,0,0.0,2,0.500000,0,0.0,4,1.0,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,3,0.750000,1,0.250000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


In [92]:
bureau_bal = pd.read_csv('../input/home-credit-default-risk/bureau_balance.csv')

In [93]:
bureau_bal.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [94]:
bureau_bal_agg_conti = agg_fn_numeric(bureau_bal, 'SK_ID_BUREAU', 'BUREAU_BAL')
bureau_bal_agg_cat = agg_fn_categorical(bureau_bal, 'SK_ID_BUREAU', 'BUREAU_BAL')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [95]:
bureau_bal_agg_conti.shape

(817395, 6)

In [96]:
bureau_bal_agg_cat.shape

(817395, 16)

In [97]:
bureau_agg_merged = bureau_bal_agg_cat.merge(bureau_bal_agg_conti, on = 'SK_ID_BUREAU', how = 'outer')

In [98]:
bureau_agg_merged

,SK_ID_BUREAU,BUREAU_BAL_STATUS_0_COUNT,BUREAU_BAL_STATUS_0_COUNT_NORM,BUREAU_BAL_STATUS_1_COUNT,BUREAU_BAL_STATUS_1_COUNT_NORM,BUREAU_BAL_STATUS_2_COUNT,BUREAU_BAL_STATUS_2_COUNT_NORM,BUREAU_BAL_STATUS_3_COUNT,BUREAU_BAL_STATUS_3_COUNT_NORM,BUREAU_BAL_STATUS_4_COUNT,BUREAU_BAL_STATUS_4_COUNT_NORM,BUREAU_BAL_STATUS_5_COUNT,BUREAU_BAL_STATUS_5_COUNT_NORM,BUREAU_BAL_STATUS_C_COUNT,BUREAU_BAL_STATUS_C_COUNT_NORM,BUREAU_BAL_STATUS_X_COUNT,BUREAU_BAL_STATUS_X_COUNT_NORM,BUREAU_BAL_MONTHS_BALANCE_COUNT,BUREAU_BAL_MONTHS_BALANCE_MEAN,BUREAU_BAL_MONTHS_BALANCE_MAX,BUREAU_BAL_MONTHS_BALANCE_MIN,BUREAU_BAL_MONTHS_BALANCE_SUM
0,5001709,0,0.000000,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,86,0.886598,11,0.113402,97,-48.0,0,-96,-4656
1,5001710,5,0.060241,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,48,0.578313,30,0.361446,83,-41.0,0,-82,-3403
2,5001711,3,0.750000,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000,1,0.250000,4,-1.5,0,-3,-6
3,5001712,10,0.526316,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,9,0.473684,0,0.000000,19,-9.0,0,-18,-171
4,5001713,0,0.000000,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000,22,1.000000,22,-10.5,0,-21,-231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817390,6842884,9,0.187500,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,20,0.416667,19,0.395833,48,-23.5,0,-47,-1128
817391,6842885,12,0.500000,0,0.000000,0,0.0,0,0.0,0,0.0,12,0.5,0,0.000000,0,0.000000,24,-11.5,0,-23,-276
817392,6842886,8,0.242424,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,25,0.757576,0,0.000000,33,-16.0,0,-32,-528
817393,6842887,6,0.162162,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,31,0.837838,0,0.000000,37,-18.0,0,-36,-666


In [101]:
temp = bureau_agg_merged.merge(bureau[['SK_ID_BUREAU', 'SK_ID_CURR']], how = 'left', on = 'SK_ID_BUREAU')

In [102]:
temp

,SK_ID_BUREAU,BUREAU_BAL_STATUS_0_COUNT,BUREAU_BAL_STATUS_0_COUNT_NORM,BUREAU_BAL_STATUS_1_COUNT,BUREAU_BAL_STATUS_1_COUNT_NORM,BUREAU_BAL_STATUS_2_COUNT,BUREAU_BAL_STATUS_2_COUNT_NORM,BUREAU_BAL_STATUS_3_COUNT,BUREAU_BAL_STATUS_3_COUNT_NORM,BUREAU_BAL_STATUS_4_COUNT,BUREAU_BAL_STATUS_4_COUNT_NORM,BUREAU_BAL_STATUS_5_COUNT,BUREAU_BAL_STATUS_5_COUNT_NORM,BUREAU_BAL_STATUS_C_COUNT,BUREAU_BAL_STATUS_C_COUNT_NORM,BUREAU_BAL_STATUS_X_COUNT,BUREAU_BAL_STATUS_X_COUNT_NORM,BUREAU_BAL_MONTHS_BALANCE_COUNT,BUREAU_BAL_MONTHS_BALANCE_MEAN,BUREAU_BAL_MONTHS_BALANCE_MAX,BUREAU_BAL_MONTHS_BALANCE_MIN,BUREAU_BAL_MONTHS_BALANCE_SUM,SK_ID_CURR
0,5001709,0,0.000000,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,86,0.886598,11,0.113402,97,-48.0,0,-96,-4656,NaN
1,5001710,5,0.060241,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,48,0.578313,30,0.361446,83,-41.0,0,-82,-3403,162368.0
2,5001711,3,0.750000,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000,1,0.250000,4,-1.5,0,-3,-6,162368.0
3,5001712,10,0.526316,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,9,0.473684,0,0.000000,19,-9.0,0,-18,-171,162368.0
4,5001713,0,0.000000,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000,22,1.000000,22,-10.5,0,-21,-231,150635.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817390,6842884,9,0.187500,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,20,0.416667,19,0.395833,48,-23.5,0,-47,-1128,387020.0
817391,6842885,12,0.500000,0,0.000000,0,0.0,0,0.0,0,0.0,12,0.5,0,0.000000,0,0.000000,24,-11.5,0,-23,-276,387020.0
817392,6842886,8,0.242424,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,25,0.757576,0,0.000000,33,-16.0,0,-32,-528,387020.0
817393,6842887,6,0.162162,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,31,0.837838,0,0.000000,37,-18.0,0,-36,-666,387020.0


In [104]:
bureau_bal_final = agg_fn_numeric(temp.drop('SK_ID_BUREAU', axis = 1), 'SK_ID_CURR', 'AGG')

In [105]:
def double_agg_fn(data, ids, names, id_df):
    agg_conti = agg_fn_numeric(data, ids[1], names[1])
    agg_cat = agg_fn_categorical(data, ids[1], names[1])
    agg_merged = agg_cat.merge(agg_conti, on = ids[1], how = 'outer')
    temp = agg_merged.merge(id_df[[ids[1], ids[0]]], how = 'left', on = ids[1])
    final = agg_fn_numeric(temp.drop(ids[1], axis = 1), ids[0], names[0])
    return final

In [107]:
double_agg_fn(bureau_bal, ['SK_ID_CURR', 'SK_ID_BUREAU'], ['AGG', 'BURAEU_BAL'], bureau)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,SK_ID_CURR,AGG_BURAEU_BAL_STATUS_0_COUNT_COUNT,AGG_BURAEU_BAL_STATUS_0_COUNT_MEAN,AGG_BURAEU_BAL_STATUS_0_COUNT_MAX,AGG_BURAEU_BAL_STATUS_0_COUNT_MIN,AGG_BURAEU_BAL_STATUS_0_COUNT_SUM,AGG_BURAEU_BAL_STATUS_0_COUNT_NORM_COUNT,AGG_BURAEU_BAL_STATUS_0_COUNT_NORM_MEAN,AGG_BURAEU_BAL_STATUS_0_COUNT_NORM_MAX,AGG_BURAEU_BAL_STATUS_0_COUNT_NORM_MIN,AGG_BURAEU_BAL_STATUS_0_COUNT_NORM_SUM,AGG_BURAEU_BAL_STATUS_1_COUNT_COUNT,AGG_BURAEU_BAL_STATUS_1_COUNT_MEAN,AGG_BURAEU_BAL_STATUS_1_COUNT_MAX,AGG_BURAEU_BAL_STATUS_1_COUNT_MIN,AGG_BURAEU_BAL_STATUS_1_COUNT_SUM,AGG_BURAEU_BAL_STATUS_1_COUNT_NORM_COUNT,AGG_BURAEU_BAL_STATUS_1_COUNT_NORM_MEAN,AGG_BURAEU_BAL_STATUS_1_COUNT_NORM_MAX,AGG_BURAEU_BAL_STATUS_1_COUNT_NORM_MIN,AGG_BURAEU_BAL_STATUS_1_COUNT_NORM_SUM,AGG_BURAEU_BAL_STATUS_2_COUNT_COUNT,AGG_BURAEU_BAL_STATUS_2_COUNT_MEAN,AGG_BURAEU_BAL_STATUS_2_COUNT_MAX,AGG_BURAEU_BAL_STATUS_2_COUNT_MIN,AGG_BURAEU_BAL_STATUS_2_COUNT_SUM,AGG_BURAEU_BAL_STATUS_2_COUNT_NORM_COUNT,AGG_BURAEU_BAL_STATUS_2_COUNT_NORM_MEAN,AGG_BURAEU_BAL_STATUS_2_COUNT_NORM_MAX,AGG_BURAEU_BAL_STATUS_2_COUNT_NORM_MIN,AGG_BURAEU_BAL_STATUS_2_COUNT_NORM_SUM,AGG_BURAEU_BAL_STATUS_3_COUNT_COUNT,AGG_BURAEU_BAL_STATUS_3_COUNT_MEAN,AGG_BURAEU_BAL_STATUS_3_COUNT_MAX,AGG_BURAEU_BAL_STATUS_3_COUNT_MIN,AGG_BURAEU_BAL_STATUS_3_COUNT_SUM,AGG_BURAEU_BAL_STATUS_3_COUNT_NORM_COUNT,AGG_BURAEU_BAL_STATUS_3_COUNT_NORM_MEAN,AGG_BURAEU_BAL_STATUS_3_COUNT_NORM_MAX,AGG_BURAEU_BAL_STATUS_3_COUNT_NORM_MIN,AGG_BURAEU_BAL_STATUS_3_COUNT_NORM_SUM,AGG_BURAEU_BAL_STATUS_4_COUNT_COUNT,AGG_BURAEU_BAL_STATUS_4_COUNT_MEAN,AGG_BURAEU_BAL_STATUS_4_COUNT_MAX,AGG_BURAEU_BAL_STATUS_4_COUNT_MIN,AGG_BURAEU_BAL_STATUS_4_COUNT_SUM,AGG_BURAEU_BAL_STATUS_4_COUNT_NORM_COUNT,AGG_BURAEU_BAL_STATUS_4_COUNT_NORM_MEAN,AGG_BURAEU_BAL_STATUS_4_COUNT_NORM_MAX,AGG_BURAEU_BAL_STATUS_4_COUNT_NORM_MIN,AGG_BURAEU_BAL_STATUS_4_COUNT_NORM_SUM,AGG_BURAEU_BAL_STATUS_5_COUNT_COUNT,AGG_BURAEU_BAL_STATUS_5_COUNT_MEAN,AGG_BURAEU_BAL_STATUS_5_COUNT_MAX,AGG_BURAEU_BAL_STATUS_5_COUNT_MIN,AGG_BURAEU_BAL_STATUS_5_COUNT_SUM,AGG_BURAEU_BAL_STATUS_5_COUNT_NORM_COUNT,AGG_BURAEU_BAL_STATUS_5_COUNT_NORM_MEAN,AGG_BURAEU_BAL_STATUS_5_COUNT_NORM_MAX,AGG_BURAEU_BAL_STATUS_5_COUNT_NORM_MIN,AGG_BURAEU_BAL_STATUS_5_COUNT_NORM_SUM,AGG_BURAEU_BAL_STATUS_C_COUNT_COUNT,AGG_BURAEU_BAL_STATUS_C_COUNT_MEAN,AGG_BURAEU_BAL_STATUS_C_COUNT_MAX,AGG_BURAEU_BAL_STATUS_C_COUNT_MIN,AGG_BURAEU_BAL_STATUS_C_COUNT_SUM,AGG_BURAEU_BAL_STATUS_C_COUNT_NORM_COUNT,AGG_BURAEU_BAL_STATUS_C_COUNT_NORM_MEAN,AGG_BURAEU_BAL_STATUS_C_COUNT_NORM_MAX,AGG_BURAEU_BAL_STATUS_C_COUNT_NORM_MIN,AGG_BURAEU_BAL_STATUS_C_COUNT_NORM_SUM,AGG_BURAEU_BAL_STATUS_X_COUNT_COUNT,AGG_BURAEU_BAL_STATUS_X_COUNT_MEAN,AGG_BURAEU_BAL_STATUS_X_COUNT_MAX,AGG_BURAEU_BAL_STATUS_X_COUNT_MIN,AGG_BURAEU_BAL_STATUS_X_COUNT_SUM,AGG_BURAEU_BAL_STATUS_X_COUNT_NORM_COUNT,AGG_BURAEU_BAL_STATUS_X_COUNT_NORM_MEAN,AGG_BURAEU_BAL_STATUS_X_COUNT_NORM_MAX,AGG_BURAEU_BAL_STATUS_X_COUNT_NORM_MIN,AGG_BURAEU_BAL_STATUS_X_COUNT_NORM_SUM,AGG_BURAEU_BAL_MONTHS_BALANCE_COUNT_COUNT,AGG_BURAEU_BAL_MONTHS_BALANCE_COUNT_MEAN,AGG_BURAEU_BAL_MONTHS_BALANCE_COUNT_MAX,AGG_BURAEU_BAL_MONTHS_BALANCE_COUNT_MIN,AGG_BURAEU_BAL_MONTHS_BALANCE_COUNT_SUM,AGG_BURAEU_BAL_MONTHS_BALANCE_MEAN_COUNT,AGG_BURAEU_BAL_MONTHS_BALANCE_MEAN_MEAN,AGG_BURAEU_BAL_MONTHS_BALANCE_MEAN_MAX,AGG_BURAEU_BAL_MONTHS_BALANCE_MEAN_MIN,AGG_BURAEU_BAL_MONTHS_BALANCE_MEAN_SUM,AGG_BURAEU_BAL_MONTHS_BALANCE_MAX_COUNT,AGG_BURAEU_BAL_MONTHS_BALANCE_MAX_MEAN,AGG_BURAEU_BAL_MONTHS_BALANCE_MAX_MAX,AGG_BURAEU_BAL_MONTHS_BALANCE_MAX_MIN,AGG_BURAEU_BAL_MONTHS_BALANCE_MAX_SUM,AGG_BURAEU_BAL_MONTHS_BALANCE_MIN_COUNT,AGG_BURAEU_BAL_MONTHS_BALANCE_MIN_MEAN,AGG_BURAEU_BAL_MONTHS_BALANCE_MIN_MAX,AGG_BURAEU_BAL_MONTHS_BALANCE_MIN_MIN,AGG_BURAEU_BAL_MONTHS_BALANCE_MIN_SUM,AGG_BURAEU_BAL_MONTHS_BALANCE_SUM_COUNT,AGG_BURAEU_BAL_MONTHS_BALANCE_SUM_MEAN,AGG_BURAEU_BAL_MONTHS_BALANCE_SUM_MAX,AGG_BURAEU_BAL_MONTHS_BALANCE_SUM_MIN,AGG_BURAEU_BAL_MONTHS_BALANCE_SUM_SUM
0,100001.0,7,4.428571,12,1,31.0,7,0.3366

In [108]:
bureau_bal_final

,SK_ID_CURR,AGG_BUREAU_BAL_STATUS_0_COUNT_COUNT,AGG_BUREAU_BAL_STATUS_0_COUNT_MEAN,AGG_BUREAU_BAL_STATUS_0_COUNT_MAX,AGG_BUREAU_BAL_STATUS_0_COUNT_MIN,AGG_BUREAU_BAL_STATUS_0_COUNT_SUM,AGG_BUREAU_BAL_STATUS_0_COUNT_NORM_COUNT,AGG_BUREAU_BAL_STATUS_0_COUNT_NORM_MEAN,AGG_BUREAU_BAL_STATUS_0_COUNT_NORM_MAX,AGG_BUREAU_BAL_STATUS_0_COUNT_NORM_MIN,AGG_BUREAU_BAL_STATUS_0_COUNT_NORM_SUM,AGG_BUREAU_BAL_STATUS_1_COUNT_COUNT,AGG_BUREAU_BAL_STATUS_1_COUNT_MEAN,AGG_BUREAU_BAL_STATUS_1_COUNT_MAX,AGG_BUREAU_BAL_STATUS_1_COUNT_MIN,AGG_BUREAU_BAL_STATUS_1_COUNT_SUM,AGG_BUREAU_BAL_STATUS_1_COUNT_NORM_COUNT,AGG_BUREAU_BAL_STATUS_1_COUNT_NORM_MEAN,AGG_BUREAU_BAL_STATUS_1_COUNT_NORM_MAX,AGG_BUREAU_BAL_STATUS_1_COUNT_NORM_MIN,AGG_BUREAU_BAL_STATUS_1_COUNT_NORM_SUM,AGG_BUREAU_BAL_STATUS_2_COUNT_COUNT,AGG_BUREAU_BAL_STATUS_2_COUNT_MEAN,AGG_BUREAU_BAL_STATUS_2_COUNT_MAX,AGG_BUREAU_BAL_STATUS_2_COUNT_MIN,AGG_BUREAU_BAL_STATUS_2_COUNT_SUM,AGG_BUREAU_BAL_STATUS_2_COUNT_NORM_COUNT,AGG_BUREAU_BAL_STATUS_2_COUNT_NORM_MEAN,AGG_BUREAU_BAL_STATUS_2_COUNT_NORM_MAX,AGG_BUREAU_BAL_STATUS_2_COUNT_NORM_MIN,AGG_BUREAU_BAL_STATUS_2_COUNT_NORM_SUM,AGG_BUREAU_BAL_STATUS_3_COUNT_COUNT,AGG_BUREAU_BAL_STATUS_3_COUNT_MEAN,AGG_BUREAU_BAL_STATUS_3_COUNT_MAX,AGG_BUREAU_BAL_STATUS_3_COUNT_MIN,AGG_BUREAU_BAL_STATUS_3_COUNT_SUM,AGG_BUREAU_BAL_STATUS_3_COUNT_NORM_COUNT,AGG_BUREAU_BAL_STATUS_3_COUNT_NORM_MEAN,AGG_BUREAU_BAL_STATUS_3_COUNT_NORM_MAX,AGG_BUREAU_BAL_STATUS_3_COUNT_NORM_MIN,AGG_BUREAU_BAL_STATUS_3_COUNT_NORM_SUM,AGG_BUREAU_BAL_STATUS_4_COUNT_COUNT,AGG_BUREAU_BAL_STATUS_4_COUNT_MEAN,AGG_BUREAU_BAL_STATUS_4_COUNT_MAX,AGG_BUREAU_BAL_STATUS_4_COUNT_MIN,AGG_BUREAU_BAL_STATUS_4_COUNT_SUM,AGG_BUREAU_BAL_STATUS_4_COUNT_NORM_COUNT,AGG_BUREAU_BAL_STATUS_4_COUNT_NORM_MEAN,AGG_BUREAU_BAL_STATUS_4_COUNT_NORM_MAX,AGG_BUREAU_BAL_STATUS_4_COUNT_NORM_MIN,AGG_BUREAU_BAL_STATUS_4_COUNT_NORM_SUM,AGG_BUREAU_BAL_STATUS_5_COUNT_COUNT,AGG_BUREAU_BAL_STATUS_5_COUNT_MEAN,AGG_BUREAU_BAL_STATUS_5_COUNT_MAX,AGG_BUREAU_BAL_STATUS_5_COUNT_MIN,AGG_BUREAU_BAL_STATUS_5_COUNT_SUM,AGG_BUREAU_BAL_STATUS_5_COUNT_NORM_COUNT,AGG_BUREAU_BAL_STATUS_5_COUNT_NORM_MEAN,AGG_BUREAU_BAL_STATUS_5_COUNT_NORM_MAX,AGG_BUREAU_BAL_STATUS_5_COUNT_NORM_MIN,AGG_BUREAU_BAL_STATUS_5_COUNT_NORM_SUM,AGG_BUREAU_BAL_STATUS_C_COUNT_COUNT,AGG_BUREAU_BAL_STATUS_C_COUNT_MEAN,AGG_BUREAU_BAL_STATUS_C_COUNT_MAX,AGG_BUREAU_BAL_STATUS_C_COUNT_MIN,AGG_BUREAU_BAL_STATUS_C_COUNT_SUM,AGG_BUREAU_BAL_STATUS_C_COUNT_NORM_COUNT,AGG_BUREAU_BAL_STATUS_C_COUNT_NORM_MEAN,AGG_BUREAU_BAL_STATUS_C_COUNT_NORM_MAX,AGG_BUREAU_BAL_STATUS_C_COUNT_NORM_MIN,AGG_BUREAU_BAL_STATUS_C_COUNT_NORM_SUM,AGG_BUREAU_BAL_STATUS_X_COUNT_COUNT,AGG_BUREAU_BAL_STATUS_X_COUNT_MEAN,AGG_BUREAU_BAL_STATUS_X_COUNT_MAX,AGG_BUREAU_BAL_STATUS_X_COUNT_MIN,AGG_BUREAU_BAL_STATUS_X_COUNT_SUM,AGG_BUREAU_BAL_STATUS_X_COUNT_NORM_COUNT,AGG_BUREAU_BAL_STATUS_X_COUNT_NORM_MEAN,AGG_BUREAU_BAL_STATUS_X_COUNT_NORM_MAX,AGG_BUREAU_BAL_STATUS_X_COUNT_NORM_MIN,AGG_BUREAU_BAL_STATUS_X_COUNT_NORM_SUM,AGG_BUREAU_BAL_MONTHS_BALANCE_COUNT_COUNT,AGG_BUREAU_BAL_MONTHS_BALANCE_COUNT_MEAN,AGG_BUREAU_BAL_MONTHS_BALANCE_COUNT_MAX,AGG_BUREAU_BAL_MONTHS_BALANCE_COUNT_MIN,AGG_BUREAU_BAL_MONTHS_BALANCE_COUNT_SUM,AGG_BUREAU_BAL_MONTHS_BALANCE_MEAN_COUNT,AGG_BUREAU_BAL_MONTHS_BALANCE_MEAN_MEAN,AGG_BUREAU_BAL_MONTHS_BALANCE_MEAN_MAX,AGG_BUREAU_BAL_MONTHS_BALANCE_MEAN_MIN,AGG_BUREAU_BAL_MONTHS_BALANCE_MEAN_SUM,AGG_BUREAU_BAL_MONTHS_BALANCE_MAX_COUNT,AGG_BUREAU_BAL_MONTHS_BALANCE_MAX_MEAN,AGG_BUREAU_BAL_MONTHS_BALANCE_MAX_MAX,AGG_BUREAU_BAL_MONTHS_BALANCE_MAX_MIN,AGG_BUREAU_BAL_MONTHS_BALANCE_MAX_SUM,AGG_BUREAU_BAL_MONTHS_BALANCE_MIN_COUNT,AGG_BUREAU_BAL_MONTHS_BALANCE_MIN_MEAN,AGG_BUREAU_BAL_MONTHS_BALANCE_MIN_MAX,AGG_BUREAU_BAL_MONTHS_BALANCE_MIN_MIN,AGG_BUREAU_BAL_MONTHS_BALANCE_MIN_SUM,AGG_BUREAU_BAL_MONTHS_BALANCE_SUM_COUNT,AGG_BUREAU_BAL_MONTHS_BALANCE_SUM_MEAN,AGG_BUREAU_BAL_MONTHS_BALANCE_SUM_MAX,AGG_BUREAU_BAL_MONTHS_BALANCE_SUM_MIN,AGG_BUREAU_BAL_MONTHS_BALANCE_SUM_SUM
0,100001.0,7,4.428571,12,1,31.0,7,0.3366

In [110]:
pos_cash = pd.read_csv('../input/home-credit-default-risk/POS_CASH_balance.csv')

In [111]:
pos_cash

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0
...,...,...,...,...,...,...,...,...
10001353,2448283,226558,-20,6.0,0.0,Active,843,0
10001354,1717234,141565,-19,12.0,0.0,Active,602,0
10001355,1283126,315695,-21,10.0,0.0,Active,609,0
10001356,1082516,450255,-22,12.0,0.0,Active,614,0


In [ ]:
double_agg_fn(pos_cash, ['SK_ID_CURR', 'SK_ID_PREV'], ['AGG', 'POS_CASH_BAL'], pos_cash)